In [5]:
# setup vars/funcs for scanCore


# This is for storing vars without code modification
try:
    from privVars import *
except ImportError:
    DSICORD_WEBHOOK = "https://discordapp.com/api/webhooks/..."
    MONGO_URL = "mongodb+srv://..."
    # add other vars here


def dprint(text, end="\r"):
    if DEBUG:
        print(text, end=end)


def disLog(text, end="\r"):
    try:
        import requests

        url = DSICORD_WEBHOOK
        data = {"content": text + end}
        requests.post(url, data=data)
    except Exception as e:
        pass


pingsPerSec = 1200
maxActive = 12
DEBUG = True


In [6]:
#   creates ip ranges

import random

# create a list of ipv4 ranges
ip_lists = []
for i in range(255):
    for j in range(255):
        ip_lists.append(f"{i}.{j}.0.0/16")
random.shuffle(ip_lists)

ip_lists = ip_lists[:200]  # remove for final version

In [7]:
#     checks if the ip is runnging a server

import time

upHosts = []
results = []
start = time.time()


def scan(ip_list):
    try:
        import masscan
        import json

        scanner = masscan.PortScanner()
        scanner.scan(
            ip_list,
            ports="25565",
            arguments="--max-rate {}".format(pingsPerSec / maxActive),
        )
        res = json.loads(scanner.scan_result)
        global results
        results.append(list(res["scan"].keys()))
    except Exception as e:
        print(e)


# -----------------------------

import threading

print("Starting threads")
threads = []
try:
    for host in ip_lists:
        while True:  # wait for a thread to finish
            c = 0
            for i in threads:
                if i.is_alive():
                    c += 1
            if c <= maxActive:
                break

            time.sleep(0.1)

        t = threading.Thread(target=scan, args=(host,))
        t.start()
        threads.append(t)

        c = 0
        for result in results:
            c += len(result)
        if ip_lists.index(host) % 10 == 0:
            print(
                f"\r\rStarted thread for {host} | Total Threads: {len(threads)} | Total Found {c} | {round(len(threads)/len(ip_lists),4)*100}% done{' '*20}",
                end="",
            )
            disLog(
                f"Started thread for {host} | Total Threads: {len(threads)} | Total Found {c} | {round(len(threads)/len(ip_lists),4)*100}% done"
            )
except KeyboardInterrupt:
    print("\nMonitoring threads")

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break


c = 0
for result in results:
    c += len(result)
dur = str((time.time() - start) / (len(threads)) / 60).split(".")

print(f"\nfound {c} hosts | average time/list {dur[0]}min {' '*40}")
disLog(f"found {c} hosts | average time/list {dur[0]}min")


Starting threads
Started thread for 119.26.0.0/16 | Total Threads: 191 | Total Found 2704 | 95.5% done                                
found 2732 hosts | average time/list 1min                                         


In [8]:
#     checks if the ip is runnging a minecraft server

import time


start_time = time.time()
res = []
lst = []
for result in results:
    for host in result:
        lst.append(host)


def check(host):
    try:
        import mcstatus
        import re

        print("\r", end="")
        server = mcstatus.JavaServer.lookup(host)
        status = server.status()
        description = status.description
        # remove color codes and whitespace
        description = description = re.sub(r"§\S*[|]*\s*", "", description)

        res.append(host)
    except Exception:
        pass


# -----------------------------

import threading

print("Starting threads")
threads = []
for host in lst:
    thread = threading.Thread(target=check, args=(host,))
    time.sleep(0.1)
    thread.start()
    threads.append(thread)
    print(
        f"\r\rStarted thread for {host} | Total Threads: {len(threads)} | Total Found {len(res)}{' '*20}",
        end="",
    )

print("Monitoring threads")
# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break


Starting threads
Started thread for 93.202.34.208 | Total Threads: 2732 | Total Found 103                    Monitoring threads


In [9]:
#   Deal with the data
print(f"found {len(res)} servers")
disLog(f"found {len(res)} servers")


found 107 servers


In [10]:
# database stuff

import pymongo
import mcstatus

client = pymongo.MongoClient(MONGO_URL, server_api=pymongo.server_api.ServerApi("1"))  # type: ignore

db = client["mc"]
col = db["servers"]
c = 0

for host in res:
    if not col.find_one({"host": host}):
        try:
            import re

            server = mcstatus.JavaServer.lookup(host)

            description = server.status().description
            description = re.sub(r"§\S*[|]*\s*", "", description)

            if server.status().players.sample is not None:
                players = list(i.name for i in server.status().players.sample) # type: ignore
            else:
                players = []

            data = {
                "host": host,
                "lastOnline": time.time(),
                "lastOnlinePlayers": server.status().players.online,
                "lastOnlineVersion": str(server.status().version.name),
                "lastOnlineDescription": str(description),
                "lastOnlinePing": int(server.status().latency * 10),
                "lastOnlinePlayersList": players,
                "lastOnlinePlayersMax": server.status().players.max,
                "lastOnlineVersionProtocol": str(server.status().version.protocol),
            }

            col.insert_one(data)
            print(f"added {host}")
            c += 1
        except Exception as e:
            print("\r", e, " | ", host, end="\r")
            pass

print(f"\r\radded {c} new servers to database", end="")
disLog(f"added {c} new servers to database")


added 77.223.34.211
added 71.237.9.223
added 71.237.152.123
added 71.237.9.113
added 71.237.129.1957.143.102
added 71.237.227.137
added 71.237.50.78
added 71.237.60.229
added 71.237.201.227
added 71.237.23.70
added 71.237.227.98
added 71.237.93.182
added 38.34.56.23
added 38.34.203.122
added 49.158.1.233
added 49.158.21.251
added 49.158.232.59
added 49.158.145.87
added 106.160.10.22
added 106.160.90.156
added 106.160.170.38
added 205.161.23.103
added 95.119.72.17
added 188.93.190.112
added 193.189.117.27
added 88.172.96.17
added 107.220.136.29
added 107.220.161.206
added 107.220.195.152
added 107.220.137.154
added 107.220.159.241
added 107.220.19.250
added 107.220.237.88
added 107.220.19.75
added 49.254.130.187
added 49.254.130.168
added 117.50.176.123
added 117.50.163.179
added 117.50.163.106
added 117.50.178.142
added 117.50.190.64
added 117.50.181.77
added 117.50.182.235
added 117.50.179.228
added 117.50.190.42
added 117.50.163.125
added 117.50.163.239
added 203.190.22.27
added 91.1

In [13]:
# search for host 172.65.103.82

host = "172.65.103.82"

print(col.find_one({"host": host}))

server = mcstatus.JavaServer.lookup(host)
print(server.status().description)
print(server.status().version.name)
print(server.status().version.protocol)
print(server.status().players.online)
print(server.status().players.max)
print(server.status().players.sample)
print(server.status().latency)

{'_id': ObjectId('636d6a445058435d4f78aad5'), 'host': '172.65.103.82', 'lastOnline': 1668115005.943593, 'lastOnlinePlayers': 1, 'lastOnlineVersion': 'Paper 1.19.2', 'lastOnlineDescription': "I don't start shit, but I can tell you how it ends Don't get sad, get even So on the weekends I don't dress for friends Lately, I've been dressin' for revenge", 'lastOnlinePing': 0.9141623110044748, 'lastOnlinePlayersList': '[<mcstatus.pinger.PingResponse.Players.Player object at 0x7fb05cfbfbb0>]', 'lastOnlinePlayersMax': 10, 'lastOnlineVersionProtocol': '760'}
I don't start shit, but I can tell you how it ends Don't get sad, get even So on the weekends I don't dress for friends Lately, I've been dressin' for revenge
Paper 1.19.2
760
0
10
None
878.9870170003269
